In [1]:
import pandas as pd
import os.path

def load_(file_1, file_2):
    dct_n = {'0000':'', 'xxxx':''}
    dct = {}
    data = pd.read_excel(file_1)
    col = list(data.columns)
    for i in range(len(col)):
        atr = list(data[col[i]])
        for grade, value in enumerate(atr):
            if pd.isna(value) == False:
                cls = value.split('/')
                num, nme =  str(grade+1) + str(i+1) + cls[1], col[i] + '/' + cls[0]
                dct[num] = nme
                
    room = pd.read_excel(file_2, sheet_name=None)
    room_name = list(room.keys())

    return list(dct.keys()), dct|dct_n, room, room_name, grade+1

def preprocess_(serial, grade, max):
    if grade == 0:
        return serial
    cnt = 0
    for i in serial:
        if i[0] == str(grade):
            cnt += 1
    if cnt < max:
        dummy = [str(grade) + '000']
        serial += dummy * (max-cnt)
    elif cnt > max:
        raise ValueError(f'{grade}학년의 강의 수가 {max}개를 초과하여 계산이 불가합니다.')
    preprocess_(serial, grade-1, max)

lecture = []
def search_(serial, grade):
    if len(serial) == 0:
        return

    temp = []
    for n in range(grade):
        for num, value in enumerate(serial):
            if value[1] not in (k[1] for k in temp):
                if value[0] == str(n+1):
                    temp.append(serial.pop(num))
                    break

    if len(temp) == grade:
        lecture.append(temp)
    else:
        serial.extend(temp)
        for value_0 in enumerate(lecture):
            for value_1 in enumerate(serial):
                if value_1[1] not in (j[1] for j in value_0):
                    for value_2 in enumerate(value_0):
                        if value_2[0] == value_1[0]:
                            value_2, value_1 = value_1, value_2

    search_(lst_lecture, grade)

last = []
def put_positon_(box, lst, _d12, _d34, _d05):
    checker = ['0000', '0000', '0000']
    nkt = []
    if box: 
        chosen = box.pop()
        cholen = len(chosen)
        if cholen <= 12: chosen += ['0000'] * (12-cholen)
        else: last.append((chosen))

        key = False
        for i in ((0,3),(3,0)):
            if lst[_d12+i[0]:_d12+i[0]+3] == checker:
                lst[_d12+i[0]:_d12+i[0]+3] = chosen[0:3]
                del chosen[0:3]
                key = False
            else: key = True

            if key == False and lst[_d34+i[1]:_d34+i[1]+3] == checker:
                lst[_d34+i[1]:_d34+i[1]+3] = chosen[0:3]
                del chosen[0:3]

        if len(chosen) != 0:
            if sum(map(int, chosen)) > 0:
                for k in (0,3):
                    if lst[_d05+k:_d05+k+3] == checker and chosen:
                        lst[_d05+k:_d05+k+3] = chosen[0:3]
                        del chosen[0:3]
            else: chosen = []
            
        if len(chosen) != 0:
            last.append(chosen)

def set_postion_(lst, box):
    table_N = [[],[],[],[],[]]
    for j in range(0,18,3):
        _d1, _d2, _d3, _d4, _d5 = j, j+18, j+36, j+54, j+72
        if j % 2 == 0:
            for rp in range(2):
                value_0, value_1 = locals()['_d'+str(rp+1)], locals()['_d'+str(rp+3)]
                put_positon_(box, lst, value_0, value_1, _d5)

        for i in range(5):
            value = locals()['_d'+str(i+1)]
            table_N[i].extend(lst[value:value+3])
            if j == 3:
                table_N[i].extend(['xxxx', 'xxxx'])

    return table_N

table_raw = []
table_label = []
def make_table_(num):
    noc = len(lecture) - num
    if noc == grade:
        return

    box = [[a] * (int(a[2:])//5) if int(a[2:]) != 0 else ['0000', '0000', '0000', '0000', '0000', '0000'] for a in lecture[noc]]
    lst = room[room_name[noc]].loc[0:19].fillna('0000').drop([6,7]).T.values.tolist()
    lst = [s for i in lst for s in i]

    table_A = set_postion_(lst, box)
    table_raw.append(table_A)
    table_B = [[dct_lecture[serial] for serial in d] for d in table_A]

    df = pd.DataFrame(table_B, index=['월','화','수','목','금']).T

    df.to_excel(writer, sheet_name=room_name[noc], index=False)
    table_label.append(table_B)
    
    make_table_(num-1)

# __제반조건__01__ : 교실의 수가 학년의 수와 일치해야한다. 
# __제반조건__02__ : 강의 시간의 총합이 교실 시간 자원의 총합을 넘겨선 안된다. 

if __name__ == '__main__':
    max_lec = 5 # 한 학년당 최대 강좌 수
    file_name = 'ttt.xlsx' # 결과 파일 이름
    path_lecture = '/mnt/c/Users/MLKU/Lecture/swtk/_table_upgrade/Files/lecture.xlsx' # 강의 파일
    path_room = '/mnt/c/Users/MLKU/Lecture/swtk/_table_upgrade/Files/rooms.xlsx' # 교실 파일

    lst_lecture, dct_lecture, room, room_name, grade = load_(path_lecture, path_room)
    preprocess_(lst_lecture, grade, max_lec)
    search_(lst_lecture, grade)
    lecture = list(map(list, zip(*lecture)))

    flag = os.path.isfile(file_name)
    if flag == True:
        choice = input('동일한 이름의 파일이 존재합니다.\n덮어쓰시겠습니까? [y/n]')
    if flag == False or choice == 'y':
        with pd.ExcelWriter(file_name) as writer:
            make_table_(len(lecture))
    if len(last) > 0:
        print(f'모든 강의를 교실에 할당하지 못하였습니다.\n할당에 실패한 강의는 다음과 같습니다.\n{last}')